# Test Predictions

Notebook to compute and store predictions on the operational test set.

Full-disk and patches/sector predictions are stored in separated files

# Config

In [ ]:
# !pip install tensorflow==2.15.0

In [1]:
COLAB = True

if COLAB :
  configSetup = {
      'COLAB'           : 'True',
      'PATH_ROOT_DRIVE' : '/content/drive/MyDrive/Projects/Forecast',
      'PATH_ROOT_LOCAL' : '/content/session',
      'PATH_SUNDL'      : '/content/sundl',
      'PATH_PROJECT'    : '/content/flare_limits_pcnn'
  }
  !git clone https://github.com/gfrancisco20/sundl.git
  !git clone https://github.com/gfrancisco20/flare_limits_pcnn.git
  import sys
  import re
  sys.path.append(configSetup['PATH_SUNDL'])
  sys.path.append(configSetup['PATH_PROJECT'])
  configFile = f'{configSetup["PATH_PROJECT"]}/config.py'
  with open(configFile, 'r') as file:
    content = file.read()
  for constant in configSetup.keys():
    content = re.sub(re.compile(f'{constant} = .*'), f'{constant} = \'{configSetup[constant]}\'', content)
  with open(configFile, 'w') as file:
    file.write(content)

from config import *
if COLAB:
  from sundl.utils.colab import mountDrive
  # mouting drive content in session on colab
  mountDrive()
!pip install dill

# you must request the models  files  or recompute  them with folder 1  to run this  notebook

Cloning into 'sundl'...
remote: Enumerating objects: 973, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 973 (delta 56), reused 49 (delta 25), pack-reused 885
Receiving objects: 100% (973/973), 7.14 MiB | 14.48 MiB/s, done.
Resolving deltas: 100% (483/483), done.
Cloning into 'flare_limits_pcnn'...
remote: Enumerating objects: 403, done.
remote: Counting objects: 100% (403/403), done.
remote: Compressing objects: 100% (329/329), done.
remote: Total 403 (delta 83), reused 370 (delta 54), pack-reused 0
Receiving objects: 100% (403/403), 6.87 MiB | 13.47 MiB/s, done.
Resolving deltas: 100% (83/83), done.
Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00


# Setup

In [2]:
FOLDER = PATH_RES/'Classification_PCNN_224x448x3xCV05_2024_03_28__0'# 'Results_Paper_PCNN' #

# Seleted Models from the CV resullts
modelDict = {
  'C+_mpf_Persistent_24'                                     : 'C+_Persistent',
  'C+_mpf_jpg_PTx8_RtdXall_EquiC_AW1e5D1e4_blos_24'          : 'C+_PCNN_Blos', #
  'C+_mpf_jpg_PTx8_RtdXall_EquiC_AW1e5D1e4_0193x0211x0094_24': 'C+_PCNN_EUV',
  'M+_mpf_Persistent_24'                                     : 'M+_Persistent',
  'M+_mpf_jpg_PTx8_RtdXall_LowC2_AW1e5D1e4_0193x0211x0094_24': 'M+_PCNN_Blos',
  'M+_mpf_jpg_PTx8_RtdXall_LowC2_AW1e5D1e4_blos_24'          : 'M+_PCNN_EUV',
}
modelDictRev = {modelDict[oldName] : oldName for oldName in modelDict.keys()}

ensembles = {'C+_PCNN_Both_Max'  : ['C+_PCNN_Blos','C+_PCNN_EUV'],
             'M+_PCNN_Both_Max'  : ['M+_PCNN_Blos','M+_PCNN_EUV'],
             'C+_PCNN_Both_Avg'  : ['C+_PCNN_Blos','C+_PCNN_EUV'],
             'M+_PCNN_Both_Avg'  : ['M+_PCNN_Blos','M+_PCNN_EUV']
            #  'C+_PCNN_Histo' : ['C+_PCNN_Blos','C+_PCNN_EUV','C+_Persistent'],
            #  'M+_PCNN_Histo' : ['M+_PCNN_Blos','M+_PCNN_EUV', 'M+_Persistent']
             }

# Predictions

In [3]:
%%time
from sundl.utils.colab import mountDrive, ressourcesSetAndCheck, drive2local
############################
# SETUP
############################

# overwriting CLEAN_LOCAL :
CLEAN_LOCAL = False

if CLEAN_LOCAL:
  shutil.rmtree(PATH_ROOT_LOCAL)
  os.makedirs(PATH_ROOT_LOCAL)

# checking gpu and ram ressources
# ressourcesSetAndCheck(MIXED_PREC)

############################
# DATA IMPORT
############################

FILES2TRANSFER = {'images' : (PATH_ROOT_DRIVE_DS/'Images',        # source
                              PATH_IMAGES,                        # dest
                              ['jpg/eq_blos_448','jpg/eq_0193x0211x0094_448']#, ''] # files
                              )
                  }

drive2local(FILES2TRANSFER)


images -- jpg/eq_blos_448.zip transfered in : 0 m 22.68 s
    jpg/eq_blos_448.zip unzipped in : 0 m 16.28 s

images -- jpg/eq_0193x0211x0094_448.zip transfered in : 28558973 m 54.35 s
    jpg/eq_0193x0211x0094_448.zip unzipped in : 0 m 9.91 s
CPU times: user 19 s, sys: 9.87 s, total: 28.9 s
Wall time: 1min 24s


In [4]:
%%time
import dill as pickle
from pathlib import Path
from glob import glob
import numpy as np
import tensorflow as tf

from sundl.utils.data import read_Dataframe_With_Dates
from utilsTest import custom_objects_for_model_loading

for modelName in modelDict.keys():
  # TBR
  try:
    if modelName.split('_')[2][3] in ['4']:
      # TODO : make generic to patch number.
      continue
  except:
    pass

  labelCol = modelName.split('_')[1]
  h = int(modelName.split('_')[-1])

  # #@@@@@@@@
  # print('WARNING : Rmv l20  !')
  # h=24
  # #@@@@@@@@

  pathPredFd = F_PATH_PREDS(FOLDER)/f'{modelName}_fd.csv'
  pathPredPt = F_PATH_PREDS(FOLDER)/f'{modelName}_pt.csv'
  if not pathPredPt.exists():
    print('\n\nMODEL : ',modelName)

    # loading model path and configs for availlable fold models
    modelsConfigAndPath = glob((FOLDER/f'models/{modelName}*').as_posix())
    modelFoldsPath = [m for m in modelsConfigAndPath if m[-3:]!='pkl']
    modelFoldsConfigs = [m for m in modelsConfigAndPath if m[-3:]=='pkl']
    # foldIds = [m[-5:] for m in modelFoldsPath]
    foldIds = [m.split('/')[-1].split('.')[0][-5:] for m in modelFoldsPath]
    n_fold = len(foldIds)
    if n_fold == 0:
      continue
    # for foldIdx in range(n_fold-1,-1,-1):
    #   if not Path(f'{modelFoldsPath[foldIdx]}/assets').exists():
    #     modelFoldsPath.pop(foldIdx)
    #     modelFoldsConfigs.pop(foldIdx)
    #     foldIds.pop(foldIdx)
    print('FOLDS WITH MODELS: ',foldIds)
    if len(foldIds)==0:
      continue
    with open(modelFoldsConfigs[0], 'rb') as f1:
      config = pickle.load(f1)
    configTest = config['dataset_val']
    dfTest = read_Dataframe_With_Dates(F_PATH_TEST(configTest['labelCol'],configTest['ts_off_label_hours'][0]))
    # configTest = config['dataset_val']
    configTest['dfTimeseries'] = dfTest.copy()
    configTest['samples'] = None
    configTest['cache'] = False
    configTest['shuffle'] = False
    configTest['weightByClass'] = False
    configTest['batch_size'] = 64
    configTest['epochs'] = 1
    # STOP
    # encoders consstantss **
    classTresholds = configTest['classTresholds']
    binCls = modelName[0]
    # **
    if modelName.split('_')[2] == 'Persistent' and not pathPredPt.exists():
      # dsTest , _, _, _ = build_dataset_persistent(**configTest)
      # saved pstmod not working
      dfId2History =  dfTest.copy()
      ts_off_label_hours = h
      ts_off_history_hours = -h
      dfId2History.index = dfId2History.index.shift(periods = -ts_off_label_hours, freq='H')
      input_lag = - ts_off_history_hours
      dfId2History['history'] = dfId2History[labelCol].rolling(window = f'{input_lag}H',
                                                      closed = 'left', # min_periods = int(input_lag)
                                                      ).apply(
                                                          lambda x: x[0]) # we remove first month in case of incomplete windows
      dfId2History = dfId2History[int(ts_off_label_hours/2):-int(ts_off_label_hours/2)]
      dfId2History = dfId2History.copy()

      dfId2History['pred'] = dfId2History['history'].apply(lambda x: configTest['labelEncoder'](x))
      dfId2History['label'] = dfId2History[labelCol].apply(lambda x: configTest['labelEncoder'](x))
      dfPred = dfId2History[[labelCol,'cls','label','pred']].copy()
      dfPred.to_csv(pathPredFd)
    else:
      if 'buildDsFunction' in config.keys():
        buildDs = config[f'buildDsFunction']
      else:
        # default dataset builder if not stored in config (adapt if needed)
        buildDs = builDS_image_feature
      dsTest , _, missing_file_regexp, dfSamples_corr = buildDs(**configTest)
      dfSamples_corr = dfSamples_corr.set_index('timestamp',drop = True)

      # predictionss
      predsFd = []
      patch_predsFd = []
      for foldIdx,modelFoldPath in enumerate(modelFoldsPath):
        # Full-disk predictions
        model = tf.keras.models.load_model(modelFoldPath, compile=False,  custom_objects = custom_objects_for_model_loading)

        # STOP
        if not pathPredFd.exists():
          predsFd.append(model.predict(dsTest))
        # Patches predictions
        if modelName.split('_')[3][:2] == 'PT':
          for layer in model.layers:
            if layer.name == 'time_distributed':
              patchesBlock = layer
          patches = tf.keras.Model(model.input, patchesBlock.output, name='patches')
          patch_predsFd.append(patches.predict(dsTest))
          # predsFd could be retrieved from it to avoid double computation
          del patches
        del model
      del dsTest

      # predictions to dataframes
      if not pathPredFd.exists():
        dfPred = dfSamples_corr[[f'label_{h}','cls']].copy()
        dfPred['label'] = dfPred[f'label_{h}'].apply(lambda x: configTest['labelEncoder'](x))
        dfPred['pred'] = np.zeros(len(dfPred))
        for idx,foldId in enumerate(foldIds):
          dfPred[f'pred_{foldId}'] = predsFd[idx][:,1]
          dfPred['pred'] += dfPred[f'pred_{foldId}']
        dfPred['pred'] /= (idx+1)
        dfPred.to_csv(pathPredFd)

      if modelName.split('_')[3][:2] == 'PT':
        dfPredPatches = dfSamples_corr[[f'label_{h}','cls']].copy()
        dfPredPatches['label'] = dfPredPatches[f'label_{h}'].apply(lambda x: configTest['labelEncoder'](x))
        num_ptch = patch_predsFd[0].shape[1]
        for ptchId in range(num_ptch):
          dfPredPatches[f'pred_pt{ptchId}'] = np.zeros(len(dfPredPatches))
          for idx,foldId in enumerate(foldIds):
            dfPredPatches[f'pred_pt{ptchId}_{foldId}'] = patch_predsFd[idx][:,ptchId,0]
            dfPredPatches[f'pred_pt{ptchId}']  += dfPredPatches[f'pred_pt{ptchId}_{foldId}']
          dfPredPatches[f'pred_pt{ptchId}'] /= (idx+1)
        dfPredPatches.to_csv(pathPredPt)





MODEL :  C+_mpf_Persistent_24
FOLDS WITH MODELS:  ['fd000', 'fd001', 'fd002', 'fd003', 'fd004']


MODEL :  C+_mpf_jpg_PTx8_RtdXall_EquiC_AW1e5D1e4_blos_24
FOLDS WITH MODELS:  ['fd000', 'fd001', 'fd002', 'fd003', 'fd004']
------------------------------
labels.shape (13317, 1)
------------------------------
img_size (224, 448, 3)
im.shape (224, 448)
209/209 [==============================] - 73s 323ms/step


MODEL :  C+_mpf_jpg_PTx8_RtdXall_EquiC_AW1e5D1e4_0193x0211x0094_24
FOLDS WITH MODELS:  ['fd000', 'fd001', 'fd002', 'fd003', 'fd004']
------------------------------
labels.shape (13953, 1)
------------------------------
img_size (224, 448, 3)
im.shape (224, 448, 3)
219/219 [==============================] - 76s 322ms/step


MODEL :  M+_mpf_Persistent_24
FOLDS WITH MODELS:  ['fd000', 'fd001', 'fd002', 'fd003', 'fd004']


MODEL :  M+_mpf_jpg_PTx8_RtdXall_LowC2_AW1e5D1e4_0193x0211x0094_24
FOLDS WITH MODELS:  ['fd000', 'fd001', 'fd002', 'fd003', 'fd004']
------------------------------
l

In [5]:
import dill as pickle
from pathlib import Path
from glob import glob
import numpy as np
import tensorflow as tf
from sundl.utils.data import read_Dataframe_With_Dates
print('Ensemble models prediction')
for modelName in ensembles.keys():
  ensModIsMax = False
  if modelName.split('_')[-1] in ['max','Max']:
    ensModIsMax = True
  pathPredFd =  F_PATH_PREDS(FOLDER)/f'{modelName}_fd.csv'
  pathPredPt =  F_PATH_PREDS(FOLDER)/f'{modelName}_pt.csv'
  print(modelName,pathPredPt.exists() )
  if not pathPredPt.exists():
    for pathPred in [pathPredFd,pathPredPt]:
      predTag = pathPred.as_posix().split('_')[-1][:2]
      for idx,subModelName in enumerate(ensembles[modelName]):
        subModel = read_Dataframe_With_Dates(F_PATH_PREDS(FOLDER)/f'{modelDictRev[subModelName]}_{predTag}.csv')
        if idx == 0:
          dfPred = subModel.copy()
          pred_cols = [col for col in dfPred.columns if col[:3]=='pre']
          ptcPredCols = [col for col in dfPred.columns if len(col)==len('pred_pt0')]
          num_ptc = len(ptcPredCols)
        else:
          # keeping only common dates
          tmp = subModel.copy()
          tmp = tmp[tmp.index.isin(dfPred.index)].sort_index()
          dfPred = dfPred[dfPred.index.isin(tmp.index)].sort_index()
          # averaging
          if modelDictRev[subModelName].split('_')[2] == 'Persistent':
            # empirical persistent probability
            for ptchTag in [''] + [f'_pt{ptcId}' for ptcId in range(num_ptc)]:
              c = tmp[f'change{ptchTag}'].sum()
              tot = len(tmp)
              pChange = c/ (2*tot)
              tmp[tmp[f'histo{ptchTag}']==1][f'pred{ptchTag}'] = 1 - pChange # prob of positive event
              tmp[tmp[f'histo{ptchTag}']==0][f'pred{ptchTag}'] = pChange # prob of positive event
            for col in pred_cols:
              if ensModIsMax:
                dfPred[col] = np.maximum(dfPred[col],tmp[col])
              else:
                dfPred[col] = dfPred[col] + tmp[col]
          else:
            missingCols = [col for col in pred_cols if col not in tmp.columns]
            for col in missingCols:
              tmp[col] = dfPred[col]
            if ensModIsMax:
              for col in pred_cols:
                dfPred[col] = np.maximum(dfPred[col],tmp[col])
            else:
              dfPred[pred_cols] = dfPred[pred_cols] + tmp[pred_cols]
      if not ensModIsMax:
        dfPred[pred_cols] = dfPred[pred_cols]/len(ensembles[modelName])
      dfPred = dfPred[['mpf','cls','label']+[col for col in dfPred.columns if col[:3]=='pre']]
      dfPred.to_csv(pathPred)


Ensemble models prediction
C+_PCNN_Both_Max False
M+_PCNN_Both_Max False
C+_PCNN_Both_Avg False
M+_PCNN_Both_Avg False
